In [56]:
from util import load_external_labels

# %time load_external_labels(session, Spouse, annotator_name='gold')

from snorkel.annotations import load_gold_labels
from snorkel import SnorkelSession
session = SnorkelSession()
#L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
#L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)

# L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
# L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
# gold_labels_dev = [L[0,0] if L[0,0]==1 else -1 for L in L_gold_dev]
gold_labels_dev = [L[0,0] for L in L_gold_dev]


from snorkel.learning.utils import MentionScorer

In [57]:
#gold_labels_dev = [x[0,0] for x in L_gold_dev.todense()]
#for i,L in enumerate(gold_labels_dev):
#    print(i,gold_labels_dev[i])

# gold_labels_dev = []
# for i,L in enumerate(L_gold_dev):
#     gold_labels_dev.append(L[0,0])
    
# gold_labels_test = []
# for i,L in enumerate(L_gold_test):
#     gold_labels_test.append(L[0,0])
    
# print(len(gold_labels_dev),len(gold_labels_test))
# print(gold_labels_dev.count(1),gold_labels_dev.count(-1))
# print(len(gold_labels_dev))

print(gold_labels_dev.count(1),gold_labels_dev.count(0))
print(len(gold_labels_dev))

189 2625
2814


In [ ]:
from gensim.parsing.preprocessing import STOPWORDS
import gensim.matutils as gm

from gensim.models.keyedvectors import KeyedVectors

# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
model = KeyedVectors.load_word2vec_format('../../../snorkel/tutorials/glove_w2v.txt', binary=False)  # C binary format


wordvec_unavailable= set()
def write_to_file(wordvec_unavailable):
    with open("wordvec_unavailable.txt","w") as f:
        for word in wordvec_unavailable:
            f.write(word+"\n")

def preprocess(tokens):
    btw_words = [word for word in tokens if word not in STOPWORDS]
    btw_words = [word for word in btw_words if word.isalpha()]
    return btw_words

def get_word_vectors(btw_words): # returns vector of embeddings of words
    word_vectors= []
    for word in btw_words:
        try:
            word_v = np.array(model[word])
            word_v = word_v.reshape(len(word_v),1)
            #print(word_v.shape)
            word_vectors.append(model[word])
        except:
            wordvec_unavailable.add(word)
    return word_vectors

def get_similarity(word_vectors,target_word): # sent(list of word vecs) to word similarity
    similarity = 0
    target_word_vector = 0
    try:
        target_word_vector = model[target_word]
    except:
        wordvec_unavailable.add(target_word+" t")
        return similarity
    target_word_sparse = gm.any2sparse(target_word_vector,eps=1e-09)
    for wv in word_vectors:
        wv_sparse = gm.any2sparse(wv, eps=1e-09)
        similarity = max(similarity,gm.cossim(wv_sparse,target_word_sparse))
    return similarity


In [58]:
# ####### Discrete ##########

# spouses = {'spouse', 'wife', 'husband', 'ex-wife', 'ex-husband'}
# family = {'father', 'mother', 'sister', 'brother', 'son', 'daughter',
#               'grandfather', 'grandmother', 'uncle', 'aunt', 'cousin'}
# family = family | {f + '-in-law' for f in family}
# other = {'boyfriend', 'girlfriend' 'boss', 'employee', 'secretary', 'co-worker'}

# # Helper function to get last name
# def last_name(s):
#     name_parts = s.split(' ')
#     return name_parts[-1] if len(name_parts) > 1 else None    

# def LF_husband_wife(c):
#     return (1,1) if len(spouses.intersection(get_between_tokens(c))) > 0 else (0,0)

# def LF_husband_wife_left_window(c):
#     if len(spouses.intersection(get_left_tokens(c[0], window=2))) > 0:
#         return (1,1)
#     elif len(spouses.intersection(get_left_tokens(c[1], window=2))) > 0:
#         return (1,1)
#     else:
#         return (0,0)
    
# def LF_same_last_name(c):
#     p1_last_name = last_name(c.person1.get_span())
#     p2_last_name = last_name(c.person2.get_span())
#     if p1_last_name and p2_last_name and p1_last_name == p2_last_name:
#         if c.person1.get_span() != c.person2.get_span():
#             return (1,1)
#     return (0,0)

# def LF_no_spouse_in_sentence(c):
#     return (-1,1) if np.random.rand() < 0.75 and len(spouses.intersection(c.get_parent().words)) == 0 else (0,0)

# def LF_and_married(c):
#     return (1,1) if 'and' in get_between_tokens(c) and 'married' in get_right_tokens(c) else (0,0)
    
# def LF_familial_relationship(c):
#     return (-1,1) if len(family.intersection(get_between_tokens(c))) > 0 else (0,0)

# def LF_family_left_window(c):
#     if len(family.intersection(get_left_tokens(c[0], window=2))) > 0:
#         return (-1,1)
#     elif len(family.intersection(get_left_tokens(c[1], window=2))) > 0:
#         return (-1,1)
#     else:
#         return (0,0)

# def LF_other_relationship(c):
#     return (-1,1) if len(other.intersection(get_between_tokens(c))) > 0 else (0,0)


# import bz2

# # Function to remove special characters from text
# def strip_special(s):
#     return ''.join(c for c in s if ord(c) < 128)

# # Read in known spouse pairs and save as set of tuples
# with bz2.BZ2File('data/spouses_dbpedia.csv.bz2', 'rb') as f:
#     known_spouses = set(
#         tuple(strip_special(x.decode('utf-8')).strip().split(',')) for x in f.readlines()
#     )
# # Last name pairs for known spouses
# last_names = set([(last_name(x), last_name(y)) for x, y in known_spouses if last_name(x) and last_name(y)])
    
# def LF_distant_supervision(c):
#     p1, p2 = c.person1.get_span(), c.person2.get_span()
#     return (1,1) if (p1, p2) in known_spouses or (p2, p1) in known_spouses else (0,0)

# def LF_distant_supervision_last_names(c):
#     p1, p2 = c.person1.get_span(), c.person2.get_span()
#     p1n, p2n = last_name(p1), last_name(p2)
#     return (1,1) if (p1 != p2) and ((p1n, p2n) in last_names or (p2n, p1n) in last_names) else (0,0)


# LFs = [
#     LF_distant_supervision, LF_distant_supervision_last_names, 
#     LF_husband_wife, LF_husband_wife_left_window, LF_same_last_name,
#     LF_no_spouse_in_sentence, LF_and_married, LF_familial_relationship, 
#     LF_family_left_window, LF_other_relationship
# ]

In [59]:
##### Continuous ################


import re
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)


spouses = {'spouse', 'wife', 'husband', 'ex-wife', 'ex-husband'}
family = {'father', 'mother', 'sister', 'brother', 'son', 'daughter',
              'grandfather', 'grandmother', 'uncle', 'aunt', 'cousin'}
family = family | {f + '-in-law' for f in family}
other = {'boyfriend', 'girlfriend' 'boss', 'employee', 'secretary', 'co-worker'}

# Helper function to get last name
def last_name(s):
    name_parts = s.split(' ')
    return name_parts[-1] if len(name_parts) > 1 else None    

def LF_husband_wife(c):
    global LF_Threshold
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
    for sw in spouses:
        sc=max(sc,get_similarity(word_vectors,sw))
    return (1,sc)

def LF_husband_wife_left_window(c):
    global LF_Threshold
    sc_1 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[0])))
    for sw in spouses:
        sc_1=max(sc_1,get_similarity(word_vectors,sw))
        
    sc_2 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[1])))
    for sw in spouses:
        sc_2=max(sc_2,get_similarity(word_vectors,sw))
    return(1,max(sc_1,sc_2))
    
def LF_same_last_name(c):
    p1_last_name = last_name(c.person1.get_span())
    p2_last_name = last_name(c.person2.get_span())
    if p1_last_name and p2_last_name and p1_last_name == p2_last_name:
        if c.person1.get_span() != c.person2.get_span():
            return (1,1)
    return (0,0)

def LF_no_spouse_in_sentence(c):
    return (-1,0.75) if np.random.rand() < 0.75 and len(spouses.intersection(c.get_parent().words)) == 0 else (0,0)

def LF_and_married(c):
    global LF_Threshold
    word_vectors = get_word_vectors(preprocess(get_right_tokens(c)))
    sc = get_similarity(word_vectors,'married')
    
    if 'and' in get_between_tokens(c):
        return (1,sc)
    else:
        return (0,0)

def LF_familial_relationship(c):
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
    for fw in family:
        sc=max(sc,get_similarity(word_vectors,fw))
        
    return (-1,sc) 

def LF_family_left_window(c):
    sc_1 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[0])))
    for fw in family:
        sc_1=max(sc_1,get_similarity(word_vectors,fw))
        
    sc_2 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[1])))
    for fw in family:
        sc_2=max(sc_2,get_similarity(word_vectors,fw))
        
    return (-1,max(sc_1,sc_2))

def LF_other_relationship(c):
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
    for ow in other:
        sc=max(sc,get_similarity(word_vectors,ow))
        
    return (-1,sc) 

# def LF_other_relationship_left_window(c):
#     sc = 0
#     word_vectors = get_word_vectors(preprocess(get_left_tokens(c)))
#     for ow in other:
#         sc=max(sc,get_similarity(word_vectors,ow))
#     return (-1,sc) 

import bz2

# Function to remove special characters from text
def strip_special(s):
    return ''.join(c for c in s if ord(c) < 128)

# # Read in known spouse pairs and save as set of tuples
# with bz2.BZ2File('data/spouses_dbpedia.csv.bz2', 'rb') as f:
#     known_spouses = set(
#         tuple(strip_special(x).strip().split(',')) for x in f.readlines()
#     )
# # Last name pairs for known spouses
# last_names = set([(last_name(x), last_name(y)) for x, y in known_spouses if last_name(x) and last_name(y)])
    
def LF_distant_supervision(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    return (1,1) if (p1, p2) in known_spouses or (p2, p1) in known_spouses else (0,0)

def LF_distant_supervision_last_names(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    p1n, p2n = last_name(p1), last_name(p2)
    return (1,1) if (p1 != p2) and ((p1n, p2n) in last_names or (p2n, p1n) in last_names) else (0,1)

import numpy as np

# def LF_Three_Lists_Left_Window(c):
#     global softmax_Threshold
#     c1,s1 = LF_husband_wife_left_window(c)
#     c2,s2 = LF_family_left_window(c)
#     c3,s3 = LF_other_relationship_left_window(c)
#     sc = np.array([s1,s2,s3])
#     c = [c1,c2,c3]
#     sharp_param = 1.5
#     prob_sc = np.exp(sc * sharp_param - np.max(sc))
#     prob_sc = prob_sc / np.sum(prob_sc)
#     #print 'Left:',s1,s2,s3,prob_sc
    
#     if s1==s2 or s3==s1:
#         return (0,0)
#     return c[np.argmax(prob_sc)],1

# def LF_Three_Lists_Between_Words(c):
#     global softmax_Threshold
#     c1,s1 = LF_husband_wife(c)
#     c2,s2 = LF_familial_relationship(c)
#     c3,s3 = LF_other_relationship(c)
#     sc = np.array([s1,s2,s3])
#     c = [c1,c2,c3]
#     sharp_param = 1.5
    
#     prob_sc = np.exp(sc * sharp_param - np.max(sc))
#     prob_sc = prob_sc / np.sum(prob_sc)
#     #print 'BW:',s1,s2,s3,prob_sc
#     if s1==s2 or s3==s1:
#         return (0,0)
#     return c[np.argmax(prob_sc)],1
    
LFs = [
    LF_distant_supervision, LF_distant_supervision_last_names, 
    LF_husband_wife, LF_husband_wife_left_window, LF_same_last_name,
    LF_no_spouse_in_sentence, LF_and_married, LF_familial_relationship, 
    LF_family_left_window, LF_other_relationship
]

In [49]:
''' output:

    [[[L_x1],[S_x1]],
     [[L_x2],[S_x2]],
     ......
     ......
    ]

'''
def get_L_S_Tensor(cands): 
    
    L_S = []
    for i,ci in enumerate(cands):
        L_S_ci=[]
        L=[]
        S=[]
        for LF in LFs:
            #print LF.__name__
            l,s = LF(ci)
            L.append(l)
            S.append((s+1)/2)  #to scale scores in [0,1] 
        L_S_ci.append(L)
        L_S_ci.append(S)
        L_S.append(L_S_ci) 
        if(i%500==0 and i!=0):
            print(str(i)+'data points labelled in',(time.time() - start_time)/60,'mins')
    return L_S


In [ ]:
# import matplotlib.pyplot as plt
import time
import numpy as np
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

dev_L_S = get_L_S_Tensor(dev_cands)

np.save("dev_L_S_smooth",np.array(dev_L_S))

train_L_S = get_L_S_Tensor(train_cands)
np.save("train_L_S_smooth",np.array(train_L_S))

print("--- %s seconds ---" % (time.time() - start_time))

# test_L_S = get_L_S_Tensor(test_cands)
# pkl.dump(test_L_S,open("test_L_S.p","wb"))

In [50]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

def draw2DArray(a):
    fig = plt.figure(figsize=(6, 3.2))
    ax = fig.add_subplot(111)
    ax.set_title('colorMap')
    plt.imshow(np.array(a))
    ax.set_aspect('equal')
    cax = fig.add_axes([0.12, 0.1, 0.78, 0.8])
    cax.get_xaxis().set_visible(False)
    cax.get_yaxis().set_visible(False)
    cax.patch.set_alpha(0)
    cax.set_frame_on(False)
    plt.colorbar(orientation='vertical')
    plt.show()
    
      
def report2dict(cr):
    # Parse rows
    tmp = list()
    for row in cr.split("\n"):
        parsed_row = [x for x in row.split("  ") if len(x) > 0]
        if len(parsed_row) > 0:
            tmp.append(parsed_row)
    
    # Store in dictionary
    measures = tmp[0]

    D_class_data = defaultdict(dict)
    for row in tmp[1:]:
        class_label = row[0]
        for j, m in enumerate(measures):
            D_class_data[class_label][m.strip()] = float(row[j + 1].strip())
    return pd.DataFrame(D_class_data).T

def predictAndPrint(pl):
    print("acc",accuracy_score(gold_labels_dev,pl))
#     print(precision_recall_fscore_support(true_labels,pl,average='macro'))
    print(confusion_matrix(gold_labels_dev,pl))
#     draw2DArray(confusion_matrix(gold_labels_dev,pl))
    return report2dict(classification_report(gold_labels_dev, pl))# target_names=class_names))
    


def drawPRcurve(y_test,y_score,it_no):
    
    fig = plt.figure()
    splt = fig.add_subplot(111)
    

    precision, recall, _ = precision_recall_curve(y_test, y_score,pos_label=1)

    splt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    splt.fill_between(recall, precision, step='post', alpha=0.2,
                     color='b')
    average_precision = average_precision_score(y_test, y_score)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.05])
    plt.title('{0:d} Precision-Recall curve: AP={1:0.2f}'.format(it_no,
              average_precision))
  

In [68]:
LF_l = [
    1,1,1,1,1,-1,1,-1,-1,-1
]
import numpy as np
dev_L_S = np.load("dev_L_S_smooth.npy")
train_L_S = np.load("train_L_S_smooth.npy")

# dev_L_S = np.load("dev_L_S_discrete.npy")
# train_L_S = np.load("train_L_S_discrete.npy")

test_L_S = dev_L_S
true_labels = gold_labels_dev
print(dev_L_S.shape,train_L_S.shape)

(2814, 2, 10) (22276, 2, 10)


In [69]:
#call this only once for a kernel startup
from __future__ import absolute_import, division, print_function

import tensorflow as tf
# BATCH_SIZE = 32
seed = 12

In [70]:
NoOfLFs= len(LF_l)
NoOfClasses = 2
print(len(LF_l))

10


In [79]:
## normalized model with smooth LFs

def train_nl_s(lr,ep,th,af):
    
    BATCH_SIZE = 1
    tf.reset_default_graph()


    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(test_L_S).batch(len(test_L_S))

     
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)

        next_element = iterator.get_next()
#         print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=af,\
                                 dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)

#         print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
#         print("k",k)
        l,s =  tf.unstack(next_element,axis=1)
#         print(alphas)
#         print(s)
#         print("l",l)
#         print(s.graph)

        s_ = tf.maximum(tf.subtract(s,alphas), 0)
#         print("s_",s_)

       
    
        def iskequalsy(v,s):
            out = tf.where(tf.equal(v,s),tf.ones_like(v),\
                           -tf.ones_like(v))
#             print("out",out)
            return out

        ## smooth pout
        pout = tf.map_fn(lambda c: iskequalsy(l,c)*s_ ,np.array([-1,1],dtype=np.float64),name="pout")
       
        ## discrete pout
#         pout = tf.map_fn(lambda c: iskequalsy(l,c) ,np.array([-1,1],dtype=np.float64),name="pout")

        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout,name="t_pout")
#         print("pout",pout)

#         print("t_pout",t_pout)

        t =  tf.squeeze(thetas)
#         print("t",t)
        
        def ints(y):
            ky = iskequalsy(k,y)
#             print("ky",ky)
            out1 = alphas+((tf.exp((t*ky*(1-alphas)))-1)/(t*ky))
#             print("intsy",out1)
            return out1
        
        ## discrete normalizer
#         zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),np.arange(NoOfClasses,dtype=np.float64))


        ## smooth normalizer
        zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                   np.array([-1,1],dtype=np.float64),name="zy")
    

#         zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
#                        np.array(NoOfClasses,dtype=np.float64))
        
#         print("zy",zy)
        logz = tf.log(tf.reduce_sum(zy,axis=0),name="logz")
        
#         print("logz",logz)
        tf.summary.scalar('logz', logz)
        lsp = tf.reduce_logsumexp(t_pout,axis=0)
#         print("lsp",lsp)
        tf.summary.scalar('lsp', tf.reduce_sum(lsp))


        normloss = tf.negative(tf.reduce_sum(lsp  - logz  ))
        
   
        tf.summary.scalar('un-normloss', normloss)
#         tf.summary.histogram('thetas', t)
#         tf.summary.histogram('alphas', alphas)
#         print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

#         print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)
#         print("predict",predict)

    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(normloss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        summary_merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('./summary/train',
                                      tf.get_default_graph())
        test_writer = tf.summary.FileWriter('./summary/test')

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for en in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    it = 0
                    while True:
                        sm,_,ls,t = sess.run([summary_merged,train_step,normloss,thetas])
#                         print(t)
#                         print(tl)
                        train_writer.add_summary(sm, it)
#                         if(ls<1e-5):
#                             break
                        tl = tl + ls
                        it = it + 1
                        
                except tf.errors.OutOfRangeError:
                    pass
                print(en,"loss",tl)

                sess.run(dev_init_op)
                sm,a,t,m,pl = sess.run([summary_merged,alphas,thetas,marginals,predict])
                test_writer.add_summary(sm, en)
                print(a)
                print(t)

#                 unique, counts = np.unique(pl, return_counts=True)
#                 print(dict(zip(unique, counts)))
                print("acc",accuracy_score(true_labels,pl))
                print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))
                print()

            # Initialize an iterator over the validation dataset.
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)

            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))

            print("acc",accuracy_score(true_labels,pl))

            predictAndPrint(pl)
            print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))

#             cf = confusion_matrix(true_labels,pl)
#             print(cf)
    return pl

In [86]:
for i in np.linspace(0,1,11):
    print("alpha-mean",i)
    predicted_labels=train_nl_s(0.001/len(train_L_S),5,tf.truncated_normal_initializer(1,0.1,seed),\
                           tf.truncated_normal_initializer(i,0.001,seed))

alpha-mean 0.0
0 loss 210993.73606317406
[ 0.00217413 -0.00086366  0.00105643  0.000967    0.00205571  0.00017504
  0.00265636 -0.00062412  0.00052894 -0.00028845]
[[1.11654688 0.81322154 1.00860747 0.99966979 1.10539034 1.04173494
  1.16823607 1.03211473 1.14724214 1.06563648]]
acc 0.9253731343283582
(0.3670886075949367, 0.15343915343915343, 0.21641791044776118, None)

1 loss 210636.25317858346
[ 3.17280988e-03  1.30576197e-04  2.01868672e-03  1.92918439e-03
  3.04769271e-03 -6.19591714e-05  3.62170517e-03 -1.56230230e-03
 -4.07473515e-04 -1.22611713e-03]
[[1.11554916 0.8122285  1.00765385 0.99871412 1.1044027  1.04213109
  1.16726512 1.033024   1.1481516  1.06655238]]
acc 0.9260838663823738
(0.37662337662337664, 0.15343915343915343, 0.21804511278195485, None)

2 loss 210278.973232167
[ 0.00417149  0.00112483  0.00298162  0.00289204  0.00403972 -0.00029708
  0.00458768 -0.00250162 -0.00134505 -0.00216493]
[[1.11455144 0.81123542 1.0066995  0.99775771 1.10341496 1.0425262
  1.16629358 

0 loss 171885.1931302566
[0.40217378 0.39913489 0.40102056 0.40093108 0.40205303 0.40045142
 0.40262464 0.39942533 0.40057939 0.39976129]
[[1.11655328 0.81322232 1.00867496 0.99972973 1.10541532 1.04187171
  1.16824459 1.031989   1.14711421 1.06553016]]
acc 0.9228855721393034
(0.3541666666666667, 0.17989417989417988, 0.2385964912280702, None)

1 loss 171695.14240822828
[0.40317211 0.4001277  0.40194808 0.40185848 0.40304238 0.40048759
 0.40355864 0.39853505 0.39969185 0.3988718 ]
[[1.11556217 0.81223003 1.00778844 0.99883388 1.10445252 1.04240677
  1.16728226 1.03277368 1.1478967  1.06634203]]
acc 0.9250177683013504
(0.375, 0.1746031746031746, 0.23826714801444038, None)

2 loss 171504.6952385699
[0.40417045 0.40112056 0.40287701 0.40278729 0.40403183 0.40052222
 0.40449375 0.39764241 0.39880189 0.39797994]
[[1.11457109 0.81123768 1.00690105 0.99793709 1.10348951 1.04294265
  1.16631937 1.03356103 1.14868218 1.06715661]]
acc 0.9246624022743426
(0.36470588235294116, 0.164021164021164, 0.

0 loss 158004.07635994116
[0.80178978 0.79913531 0.80032776 0.80022985 0.80131484 0.79985737
 0.80241737 0.80023174 0.80141598 0.80085236]
[[1.11707343 0.81322156 1.00951067 1.00057442 1.10624016 1.04183987
  1.16838797 1.03134808 1.14644998 1.06457373]]
acc 0.8923240938166311
(0.3323529411764706, 0.5978835978835979, 0.42722117202268434, None)

1 loss 157980.1437335581
[0.80238924 0.80012853 0.80055975 0.80045272 0.80156051 0.79930025
 0.80312747 0.80015058 0.80136757 0.80105712]
[[1.11662203 0.81222855 1.009462   1.00052521 1.1061083  1.04234269
  1.16757533 1.03148884 1.14656606 1.06442774]]
acc 0.8923240938166311
(0.3323529411764706, 0.5978835978835979, 0.42722117202268434, None)

2 loss 157956.25471070939
[0.802987   0.80112175 0.80079227 0.80067594 0.80180566 0.79874196
 0.80383489 0.80006835 0.8013183  0.80126177]
[[1.11617199 0.81123553 1.00941282 1.00047563 1.10597657 1.04284709
  1.1667632  1.03163029 1.14668268 1.06428221]]
acc 0.8923240938166311
(0.3323529411764706, 0.597883

/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


2 loss 154405.4651393359
[1.00426285 1.00000045 1.0028813  1.00000799 1.00415361 1.00349671
 1.00476839 1.00337596 1.00454441 1.00374216]
[[1.1184141  0.81253047 1.0097917  1.00062431 1.10718171 1.04179665
  1.17039526 1.0316135  1.14738094 1.06529977]]
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

3 loss 154405.4642637172
[1.00528048 1.00000042 1.0039047  1.00000835 1.00517159 1.0045171
 1.00578454 1.00439686 1.00556119 1.00476158]
[[1.11909374 0.81253047 1.01019352 1.00062431 1.1078431  1.04233188
  1.17114856 1.03212298 1.14810411 1.06588544]]
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

4 loss 154405.46289797986
[1.0062952  1.00000036 1.00492326 1.00000841 1.00618655 1.0055337
 1.00679821 1.00541381 1.00657531 1.00577754]
[[1.11988979 0.81253047 1.01079317 1.00062431 1.1086273  1.04303077
  1.17199147 1.03280351 1.14892806 1.06661932]]
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

[1.0062952  1.00000036 1.00492326 1.00000841 1.00618655 1.0055337
 1.00679821 1.00541381 1.00657531 1

/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [87]:
predicted_labels=train_nl_s(0.001/len(train_L_S),10,tf.truncated_normal_initializer(1,0.1,seed),\
                           tf.truncated_normal_initializer(0.7,0.001,seed))

0 loss 159838.21141220749
[0.70199852 0.69913478 0.70064994 0.70050632 0.70158372 0.69974812
 0.70241884 0.69989864 0.70116151 0.70031885]
[[1.11684696 0.81322209 1.00909608 1.00020124 1.10603123 1.04198871
  1.16832555 1.03148073 1.14652464 1.06476873]]
acc 0.9076048329779673
(0.3650190114068441, 0.5079365079365079, 0.4247787610619469, None)

1 loss 159805.78696832046
[0.70281529 0.70012747 0.70120386 0.70100447 0.70210101 0.69908186
 0.7031378  0.69948201 0.70085426 0.69998286]
[[1.11616003 0.81222961 1.00863201 0.99977902 1.10568825 1.04264001
  1.16744852 1.03175497 1.14671635 1.06481849]]
acc 0.9090262970859986
(0.36964980544747084, 0.5026455026455027, 0.4260089686098655, None)

2 loss 159773.3738374559
[0.70363076 0.70112018 0.70175788 0.70150161 0.70261734 0.69841559
 0.70385253 0.69906167 0.70054384 0.69964374]
[[1.1154744  0.81123709 1.00816648 0.99935546 1.10534549 1.04329149
  1.16657206 1.0320314  1.14690984 1.06487073]]
acc 0.9108031272210376
(0.376984126984127, 0.50264550

In [83]:
# for i in np.linspace(0,1,11):
#     print("alpha-mean",i)
#     predicted_labels=train_nl_s(0.001/len(train_L_S),5,tf.truncated_normal_initializer(1,0.1,12),\
#                            tf.truncated_normal_initializer(i,0.1,12))
    
predicted_labels=train_nl_s(0.001/len(train_L_S),5,tf.truncated_normal_initializer(1,0.1,12),\
                           tf.truncated_normal_initializer(0.3,0.1,12))

0 loss 177593.6089042592
[0.41854278 0.11520687 0.31047011 0.30153524 0.40736582 0.34136772
 0.47013164 0.33035577 0.44548349 0.36387167]
[[1.11655652 0.81322315 1.00867167 0.99972798 1.10542697 1.04178828
  1.16826229 1.03197667 1.14706397 1.06550782]]
acc 0.9068941009239516
(0.26143790849673204, 0.21164021164021163, 0.23391812865497075, None)

1 loss 177396.56970193452
[0.41954094 0.11619926 0.31138325 0.30244885 0.40835387 0.3413951
 0.47105875 0.32949968 0.44462813 0.36301599]
[[1.11556877 0.8122317  1.00778122 0.99882951 1.10447602 1.04224007
  1.16731925 1.0327495  1.14779659 1.06629769]]
acc 0.9086709310589908
(0.2702702702702703, 0.21164021164021163, 0.2373887240356083, None)

2 loss 177199.12666951108
[0.42053909 0.11719167 0.3122979  0.30336394 0.409342   0.34142099
 0.47198704 0.32864074 0.44376979 0.36215742]
[[1.11458114 0.81124022 1.00688941 0.99792959 1.10352504 1.04269297
  1.16637726 1.03352561 1.14853279 1.06709081]]
acc 0.9136460554371002
(0.29850746268656714, 0.2116

In [82]:
predicted_labels=train_nl_s(0.1/len(train_L_S),5,\
                            tf.truncated_normal_initializer(1,0.1,seed),\
                            tf.truncated_normal_initializer(0.9,0.001,seed))

0 loss 155264.53954891075
[0.92690352 0.9971321  0.89117171 0.89079329 0.90543897 0.98363892
 0.9542917  0.89181359 0.8937952  0.91162108]
[[1.10009277 0.72704278 1.03085428 1.02047423 1.10633782 0.96621163
  1.12864769 1.05197377 1.16535794 1.05758622]]
acc 0.8941009239516702
(0.33126934984520123, 0.5661375661375662, 0.41796875, None)

1 loss 154352.82584004718
[0.94538075 1.00003227 0.88239876 0.88170429 0.90954794 0.99722517
 0.98051568 0.88340816 0.88622291 0.92065953]
[[1.0884174  0.72649249 1.05094155 1.03924606 1.10629312 0.95459477
  1.11214221 1.07165552 1.18340155 1.05207841]]
acc 0.8948116560056859
(0.3333333333333333, 0.5661375661375662, 0.41960784313725485, None)

2 loss 154345.657013675
[0.95711568 1.00017134 0.87422725 0.87406136 0.91291265 0.99776011
 0.99031907 0.87564378 0.87869541 0.92818664]
[[1.08146765 0.72632277 1.06995853 1.0570634  1.10656614 0.94945574
  1.10669314 1.09036948 1.20057795 1.04758399]]
acc 0.8941009239516702
(0.3333333333333333, 0.576719576719576

In [81]:
predicted_labels=train_nl_s(0.1/len(train_L_S),5,\
                            tf.truncated_normal_initializer(1,0.1,seed),\
                            tf.truncated_normal_initializer(0.9,0.01,seed))

0 loss 155548.10499263214
[0.93326346 0.98037827 0.88953763 0.88821575 0.91311962 0.98554735
 0.96406621 0.89759127 0.90964637 0.91662847]
[[1.1032539  0.72197787 1.03206361 1.02162306 1.10767904 0.96823482
  1.13417441 1.05126276 1.16661986 1.05815618]]
acc 0.8855721393034826
(0.3147632311977716, 0.5978835978835979, 0.41240875912408764, None)

1 loss 154383.652287932
[0.94953699 1.00003838 0.8767362  0.87537624 0.91583377 0.99209049
 0.98585481 0.89384701 0.90618009 0.92479421]
[[1.09328744 0.7103856  1.05400886 1.04201962 1.10870954 0.92510664
  1.12145815 1.06944685 1.18536492 1.05320904]]
acc 0.8855721393034826
(0.3147632311977716, 0.5978835978835979, 0.41240875912408764, None)

2 loss 154346.07266892138
[0.96029182 1.00012677 0.86560251 0.86458486 0.91794999 0.98825132
 0.99431731 0.89279564 0.90488922 0.93167133]
[[1.08722855 0.71028976 1.07575335 1.06207778 1.11012153 0.84748815
  1.11739958 1.08540677 1.20247734 1.04934008]]
acc 0.8781094527363185
(0.3121951219512195, 0.6772486

In [99]:
## normalized model with smooth LFs + penalties

def train(lr,ep,th,af,plv=np.array([-1,1],dtype=np.float64),smooth=True,penalty=0,p3k=3):
    
    ## lr : learning rate
    ## ep : no of epochs
    ## th : thetas initializer
    ## af : alphas initializer
    ## penalty : {1,2,3} use one of the three penalties, 0: no-penalty
    ## p3k : parameter for penalty-3 
    ## smooth : flag if smooth lfs are used 
    ## make sure smooth/discrete LF data is loaded into train_L_S and test_L_S
    ## plv : all possible label values = [-1,1] for binary, 
    ##       np.arange(0,NoOfClasses) for multiclass
    
    BATCH_SIZE = 1
    tf.reset_default_graph()


    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(test_L_S).batch(len(test_L_S))

     
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)

        next_element = iterator.get_next()
#         print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=af,\
                                 dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)

#         print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
#         print("k",k)
        l,s =  tf.unstack(next_element,axis=1)
#         print(alphas)
#         print(s)
#         print("l",l)
#         print(s.graph)

        s_ = tf.maximum(tf.subtract(s,alphas), 0)
#         print("s_",s_)

       
    
        def iskequalsy(v,s):
            out = tf.where(tf.equal(v,s),tf.ones_like(v),-tf.ones_like(v))
#             print("out",out)
            return out

        if(smooth):
            pout = tf.map_fn(lambda c: iskequalsy(l,c)*s_ ,plv,name="pout")
        else:
            pout = tf.map_fn(lambda c: iskequalsy(l,c) ,plv,name="pout")

#         print("pout",pout)    

        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout,\
                           name="t_pout")
    
#         print("t_pout",t_pout)

        t =  tf.squeeze(thetas)
#         print("t",t)
        
        def ints(y):
            ky = iskequalsy(k,y)
#             print("ky",ky)
            out1 = alphas+((tf.exp((t*ky*(1-alphas)))-1)/(t*ky))
#             print("intsy",out1)
            return out1
                

        if(smooth):
            #smooth normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                           plv,name="zy")
        else:
            #discrete normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                           plv,name="zy")

    
# 
#         zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
#                        np.array(NoOfClasses,dtype=np.float64))
        
#         print("zy",zy)
        logz = tf.log(tf.reduce_sum(zy,axis=0),name="logz")
        
#         print("logz",logz)
        tf.summary.scalar('logz', logz)
        lsp = tf.reduce_logsumexp(t_pout,axis=0)
#         print("lsp",lsp)
        tf.summary.scalar('lsp', tf.reduce_sum(lsp))

        if(penalty == 1):
            normloss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                      +tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas))
        elif(penalty == 2):
            normloss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     -tf.minimum( tf.reduce_min(thetas),0.0)
        elif(penalty == 3):
             normloss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     +tf.reduce_sum(tf.log(1+tf.exp(-thetas-pk)))
        else:
             normloss = tf.negative(tf.reduce_sum(lsp  - logz  ))
   
        tf.summary.scalar('un-normloss', normloss)
#         tf.summary.histogram('thetas', t)
#         tf.summary.histogram('alphas', alphas)
#         print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

#         print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)


    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(normloss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        summary_merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('./summary/train',
                                      tf.get_default_graph())
        test_writer = tf.summary.FileWriter('./summary/test')

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for en in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    it = 0
                    while True:
                        sm,_,ls,t = sess.run([summary_merged,train_step,normloss,thetas])
#                         print(t)
#                         print(tl)
                        train_writer.add_summary(sm, it)
#                         if(ls<1e-5):
#                             break
                        tl = tl + ls
                        it = it + 1
                        
                except tf.errors.OutOfRangeError:
                    pass
                print(en,"loss",tl)

                sess.run(dev_init_op)
                sm,a,t,m,pl = sess.run([summary_merged,alphas,thetas,marginals,predict])
                test_writer.add_summary(sm, en)
                print(a)
                print(t)
                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print("acc",accuracy_score(true_labels,pl))
                print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))
                print()

            # Initialize an iterator over the validation dataset.
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)

            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))

            print("acc",accuracy_score(true_labels,pl))

            predictAndPrint(pl)
            print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))

#             cf = confusion_matrix(true_labels,pl)
#             print(cf)
    return pl

In [100]:
for i in np.linspace(0,1,11):
    print()
    print("alpha-mean",i)
    train(0.1/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                            af = tf.truncated_normal_initializer(i,0.001,seed),
                          plv=np.array([-1,1],dtype=np.float64),smooth=True,penalty=1)


alpha-mean 0.0
0 loss 194264.85326121486
[ 0.09947401  0.09596     0.09735473  0.09725704  0.09894004 -0.01030508
  0.09894431 -0.09624256 -0.09494115 -0.09586474]
[[1.01984608 0.71578729 0.91326762 0.90408304 1.00926658 1.07292658
  1.07212253 1.12601357 1.24131725 1.16021693]]
{0: 2806, 1: 8}
acc 0.9342572850035537
(0.75, 0.031746031746031744, 0.06091370558375634, None)

1 loss 165944.78653442475
[ 0.19790907  0.19411453  0.19583783  0.19573171  0.19731637  0.00760686
  0.19744465 -0.19395243 -0.1925366  -0.19354266]
[[0.92264118 0.61691555 0.81792812 0.8082795  0.9130418  1.0849647
  0.97551319 1.22139401 1.33701538 1.25683504]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)



/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


2 loss 145426.3452882983
[ 0.29704017  0.29296915  0.29497276  0.29485531  0.29646632  0.0399873
  0.29663876 -0.29182634 -0.29029167 -0.29138179]
[[0.8261613  0.51767846 0.7251858  0.71477777 0.82094729 1.08603578
  0.88155871 1.3152654  1.43122333 1.35249425]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

3 loss 129826.68522984035
[ 0.3965422   0.39207137  0.39443259  0.39429967  0.3960568   0.07599614
  0.39621449 -0.38993288 -0.38828145 -0.3894509 ]
[[0.73526719 0.41834843 0.63658532 0.62519367 0.74518257 1.08494332
  0.79750861 1.40760353 1.52378527 1.44700078]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

4 loss 117293.74879868481
[ 0.49677966  0.49105991  0.49399921  0.49384786  0.49619953  0.11179939
  0.49619988 -0.48814211 -0.48638682 -0.48762274]
[[0.7522436  0.31907066 0.55577511 0.54350837 0.74399203 1.08535089
  0.76963201 1.49848879 1.61471096 1.54024589]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

[ 0.49677966  0.49105991  0.49399921  0

/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0 loss 185781.95847627256
[0.19967327 0.19615971 0.19744814 0.19735001 0.19911576 0.09226008
 0.19894121 0.00368871 0.00499454 0.00406713]
[[1.02003556 0.7157202  0.91386131 0.90455327 1.00954362 1.07559231
  1.07237539 1.12539065 1.24081911 1.15986489]]
{0: 2806, 1: 8}
acc 0.9342572850035537
(0.75, 0.031746031746031744, 0.06091370558375634, None)

1 loss 159483.32056801888
[ 0.29834619  0.29456714  0.29622298  0.29611599  0.29773821  0.10580634
  0.29768761 -0.09462073 -0.09321636 -0.09421548]
[[0.92345526 0.61679842 0.81989008 0.80995456 0.91485833 1.09507768
  0.97699202 1.22065646 1.33651883 1.25658421]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

2 loss 139408.32185090086
[ 0.3975604   0.39348825  0.39545592  0.39533679  0.39700739  0.13221124
  0.39698284 -0.19293461 -0.19142947 -0.1925003 ]
[[0.83099436 0.51762236 0.73035262 0.71948807 0.83356164 1.10505495
  0.88993698 1.31460596 1.43088955 1.35239227]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

3 loss 

1 loss 146478.77296745367
[0.53338662 0.69595146 0.67474378 0.67138814 0.56400598 0.49732819
 0.59363951 0.30444514 0.30581848 0.30483133]
[[1.01621361 0.61717488 0.86621487 0.86592647 1.03540186 1.18550765
  1.06752881 1.2011621  1.31727085 1.24125633]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

2 loss 135427.40164752337
[0.50000098 0.79512653 0.73337827 0.72151411 0.53204293 0.49594704
 0.61037182 0.20466859 0.20609253 0.20506636]
[[1.06801229 0.51895373 0.81971805 0.82609229 1.03869969 1.25376835
  1.07146865 1.29299437 1.41013139 1.33585511]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

3 loss 125405.84016562754
[0.50000037 0.89426097 0.77755085 0.75854575 0.50000204 0.49786435
 0.61321995 0.10519558 0.10667399 0.10560908]
[[1.14167675 0.42182486 0.78115818 0.79334306 1.05738007 1.31948832
  1.1008312  1.38573543 1.50370148 1.43105489]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

4 loss 116605.14990806696
[0.5000016  0.99333059 0.81427612 0.78807

2 loss 154345.657013675
[0.95711568 1.00017134 0.87422725 0.87406136 0.91291265 0.99776011
 0.99031907 0.87564378 0.87869541 0.92818664]
[[1.08146765 0.72632277 1.06995853 1.0570634  1.10656614 0.94945574
  1.10669314 1.09036948 1.20057795 1.04758399]]
{0: 2487, 1: 327}
acc 0.8941009239516702
(0.3333333333333333, 0.5767195767195767, 0.42248062015503873, None)

3 loss 154341.1192211615
[0.96473516 1.00014065 0.87002874 0.86936931 0.91573331 0.99703065
 0.99419881 0.87566606 0.87567845 0.93511696]
[[1.0772874  0.72628999 1.0882641  1.07415257 1.10708575 0.9450955
  1.10472967 1.1084319  1.21700396 1.04402819]]
{0: 2459, 1: 355}
acc 0.8862828713574982
(0.3154929577464789, 0.5925925925925926, 0.411764705882353, None)

4 loss 154338.10484061058
[0.96998981 1.00014084 0.86469124 0.86383808 0.91809031 0.99519522
 0.9964329  0.87568057 0.87567871 0.94170222]
[[1.07469848 0.72623898 1.10667907 1.09119193 1.10790566 0.92861019
  1.10381286 1.12572771 1.23273514 1.04161097]]
{0: 2407, 1: 407}
acc

In [101]:
for i in np.linspace(0,1,11):
    print()
    print("alpha-mean",i)
    train(0.1/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                            af = tf.truncated_normal_initializer(i,0.001,seed),
                          plv=np.array([-1,1],dtype=np.float64),smooth=True,penalty=2)


alpha-mean 0.0
0 loss 194264.85326121486
[ 0.09947401  0.09596     0.09735473  0.09725704  0.09894004 -0.01030508
  0.09894431 -0.09624256 -0.09494115 -0.09586474]
[[1.01984608 0.71578729 0.91326762 0.90408304 1.00926658 1.07292658
  1.07212253 1.12601357 1.24131725 1.16021693]]
{0: 2806, 1: 8}
acc 0.9342572850035537
(0.75, 0.031746031746031744, 0.06091370558375634, None)

1 loss 165944.78653442475
[ 0.19790907  0.19411453  0.19583783  0.19573171  0.19731637  0.00760686
  0.19744465 -0.19395243 -0.1925366  -0.19354266]
[[0.92264118 0.61691555 0.81792812 0.8082795  0.9130418  1.0849647
  0.97551319 1.22139401 1.33701538 1.25683504]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)



/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


2 loss 145426.3452882983
[ 0.29704017  0.29296915  0.29497276  0.29485531  0.29646632  0.0399873
  0.29663876 -0.29182634 -0.29029167 -0.29138179]
[[0.8261613  0.51767846 0.7251858  0.71477777 0.82094729 1.08603578
  0.88155871 1.3152654  1.43122333 1.35249425]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

3 loss 129826.68522984035
[ 0.3965422   0.39207137  0.39443259  0.39429967  0.3960568   0.07599614
  0.39621449 -0.38993288 -0.38828145 -0.3894509 ]
[[0.73526719 0.41834843 0.63658532 0.62519367 0.74518257 1.08494332
  0.79750861 1.40760353 1.52378527 1.44700078]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

4 loss 117293.74879868481
[ 0.49677966  0.49105991  0.49399921  0.49384786  0.49619953  0.11179939
  0.49619988 -0.48814211 -0.48638682 -0.48762274]
[[0.7522436  0.31907066 0.55577511 0.54350837 0.74399203 1.08535089
  0.76963201 1.49848879 1.61471096 1.54024589]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

[ 0.49677966  0.49105991  0.49399921  0

/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0 loss 185781.95847627256
[0.19967327 0.19615971 0.19744814 0.19735001 0.19911576 0.09226008
 0.19894121 0.00368871 0.00499454 0.00406713]
[[1.02003556 0.7157202  0.91386131 0.90455327 1.00954362 1.07559231
  1.07237539 1.12539065 1.24081911 1.15986489]]
{0: 2806, 1: 8}
acc 0.9342572850035537
(0.75, 0.031746031746031744, 0.06091370558375634, None)

1 loss 159483.32056801888
[ 0.29834619  0.29456714  0.29622298  0.29611599  0.29773821  0.10580634
  0.29768761 -0.09462073 -0.09321636 -0.09421548]
[[0.92345526 0.61679842 0.81989008 0.80995456 0.91485833 1.09507768
  0.97699202 1.22065646 1.33651883 1.25658421]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

2 loss 139408.32185090086
[ 0.3975604   0.39348825  0.39545592  0.39533679  0.39700739  0.13221124
  0.39698284 -0.19293461 -0.19142947 -0.1925003 ]
[[0.83099436 0.51762236 0.73035262 0.71948807 0.83356164 1.10505495
  0.88993698 1.31460596 1.43088955 1.35239227]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

3 loss 

1 loss 146478.77296745367
[0.53338662 0.69595146 0.67474378 0.67138814 0.56400598 0.49732819
 0.59363951 0.30444514 0.30581848 0.30483133]
[[1.01621361 0.61717488 0.86621487 0.86592647 1.03540186 1.18550765
  1.06752881 1.2011621  1.31727085 1.24125633]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

2 loss 135427.40164752337
[0.50000098 0.79512653 0.73337827 0.72151411 0.53204293 0.49594704
 0.61037182 0.20466859 0.20609253 0.20506636]
[[1.06801229 0.51895373 0.81971805 0.82609229 1.03869969 1.25376835
  1.07146865 1.29299437 1.41013139 1.33585511]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

3 loss 125405.84016562754
[0.50000037 0.89426097 0.77755085 0.75854575 0.50000204 0.49786435
 0.61321995 0.10519558 0.10667399 0.10560908]
[[1.14167675 0.42182486 0.78115818 0.79334306 1.05738007 1.31948832
  1.1008312  1.38573543 1.50370148 1.43105489]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

4 loss 116605.14990806696
[0.5000016  0.99333059 0.81427612 0.78807

2 loss 154345.657013675
[0.95711568 1.00017134 0.87422725 0.87406136 0.91291265 0.99776011
 0.99031907 0.87564378 0.87869541 0.92818664]
[[1.08146765 0.72632277 1.06995853 1.0570634  1.10656614 0.94945574
  1.10669314 1.09036948 1.20057795 1.04758399]]
{0: 2487, 1: 327}
acc 0.8941009239516702
(0.3333333333333333, 0.5767195767195767, 0.42248062015503873, None)

3 loss 154341.1192211615
[0.96473516 1.00014065 0.87002874 0.86936931 0.91573331 0.99703065
 0.99419881 0.87566606 0.87567845 0.93511696]
[[1.0772874  0.72628999 1.0882641  1.07415257 1.10708575 0.9450955
  1.10472967 1.1084319  1.21700396 1.04402819]]
{0: 2459, 1: 355}
acc 0.8862828713574982
(0.3154929577464789, 0.5925925925925926, 0.411764705882353, None)

4 loss 154338.10484061058
[0.96998981 1.00014084 0.86469124 0.86383808 0.91809031 0.99519522
 0.9964329  0.87568057 0.87567871 0.94170222]
[[1.07469848 0.72623898 1.10667907 1.09119193 1.10790566 0.92861019
  1.10381286 1.12572771 1.23273514 1.04161097]]
{0: 2407, 1: 407}
acc

In [105]:
train(0.1/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                            af = tf.truncated_normal_initializer(1,0.01,seed),
                          plv=np.array([-1,1],dtype=np.float64),smooth=True,penalty=2)

0 loss 154423.11505296052
[1.12960963 1.00003591 1.12573894 0.98602607 1.12898605 1.12611872
 1.1328575  1.12586352 1.13134219 1.12693168]
[[1.24184085 0.80167334 1.13637017 1.01583723 1.23121741 1.16866017
  1.29117918 1.15852328 1.26928974 1.19147305]]
{0: 2608, 1: 206}
acc 0.9214641080312722
(0.4223300970873786, 0.4603174603174603, 0.440506329113924, None)

1 loss 154232.150101072
[1.23585489 1.00013351 1.2321689  0.95857505 1.23525663 1.23256117
 1.23897777 1.23232052 1.23756257 1.23333406]
[[1.35022575 0.80161894 1.24496523 1.04543615 1.33963368 1.27726402
  1.39940635 1.26714232 1.3776245  1.30003166]]
{0: 2608, 1: 206}
acc 0.9214641080312722
(0.4223300970873786, 0.4603174603174603, 0.440506329113924, None)

2 loss 153619.95126115912
[1.34002277 1.00012845 1.3364193  0.93438749 1.33943492 1.33683523
 1.34309515 1.33660159 1.34174276 1.33759002]
[[1.45561555 0.8014836  1.35043097 1.07191777 1.44503463 1.38275399
  1.50474012 1.37263794 1.48301806 1.40550518]]
{0: 2608, 1: 206}
acc

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [54]:
## normalized model with discrete LFs

def train_nl(lr,ep,th):
    
    BATCH_SIZE = 1
    tf.reset_default_graph()


    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(test_L_S).batch(len(test_L_S))

     
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)

        next_element = iterator.get_next()
        print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=tf.truncated_normal_initializer(0.3,0.1,seed),\
                                 dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)

        print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
        print("k",k)
        l,s =  tf.unstack(next_element,axis=1)
        print(alphas)
        print(s)
        print("l",l)
        print(s.graph)

        s_ = tf.maximum(tf.subtract(s,alphas), 0)
        print("s_",s_)

       
    
        def iskequalsy(v,s):
            out = tf.where(tf.equal(v,s),tf.ones_like(v),\
                           -tf.ones_like(v))
            print("out",out)
            return out

        ### discrete pout 
        pout = tf.map_fn(lambda c: iskequalsy(l,c) ,np.array([-1,1],dtype=np.float64),name="pout")
       
        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout,name="t_pout")
        print("pout",pout)

        print("t_pout",t_pout)

        t =  tf.squeeze(thetas)
        print("t",t)
        
        def ints(y):
            ky = iskequalsy(k,y)
            print("ky",ky)
            out1 = alphas+((tf.exp((t*ky*(1-alphas)))-1)/(t*ky))
            print("intsy",out1)
            return out1
        
        zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),np.arange(NoOfClasses,dtype=np.float64))

        
        print("zy",zy)
        logz = tf.log(tf.reduce_sum(zy,axis=0),name="logz")
        
        print("logz",logz)
        tf.summary.scalar('logz', logz)
        lsp = tf.reduce_logsumexp(t_pout,axis=0)
        print("lsp",lsp)
        tf.summary.scalar('lsp', tf.reduce_sum(lsp))

        
        normloss = tf.negative(tf.reduce_sum(lsp  - logz  ))


        tf.summary.scalar('un-normloss', normloss)
#         tf.summary.histogram('thetas', t)
#         tf.summary.histogram('alphas', alphas)
        print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

        print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)
        print("predict",predict)

    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(normloss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        summary_merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('./summary/train',
                                      tf.get_default_graph())
        test_writer = tf.summary.FileWriter('./summary/test')

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for en in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    it = 0
                    while True:
                        sm,_,ls,t = sess.run([summary_merged,train_step,normloss,thetas])
#                         print(t)
#                         print(tl)
                        train_writer.add_summary(sm, it)
#                         if(ls<1e-5):
#                             break
                        tl = tl + ls
                        it = it + 1
                        
                except tf.errors.OutOfRangeError:
                    pass
                print(en,"loss",tl)

                sess.run(dev_init_op)
                sm,a,t,m,pl = sess.run([summary_merged,alphas,thetas,marginals,predict])
                test_writer.add_summary(sm, en)
#                 print(a)
#                 print(t)

#                 unique, counts = np.unique(pl, return_counts=True)
#                 print(dict(zip(unique, counts)))
                print("acc",accuracy_score(true_labels,pl))
                print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))
                print()

            # Initialize an iterator over the validation dataset.
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)

            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))

            print("acc",accuracy_score(true_labels,pl))

            predictAndPrint(pl)
            print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))

#             cf = confusion_matrix(true_labels,pl)
#             print(cf)
    return pl

In [55]:
predicted_labels=train_nl(0.1/len(train_L_S),5,tf.truncated_normal_initializer(1,0.1,12))

next_element Tensor("IteratorGetNext_1:0", shape=(?, 2, 10), dtype=float64)
thetas <tf.Variable 'thetas:0' shape=(1, 10) dtype=float64_ref>
k Tensor("Const:0", shape=(10,), dtype=float64)
<tf.Variable 'alphas:0' shape=(10,) dtype=float64_ref>
Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 10), dtype=float64)
out Tensor("pout/while/Select:0", shape=(?, 10), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(10,), dtype=float64)
out Tensor("map/while/Select:0", shape=(10,), dtype=float64)
zy Tensor("map/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
normloss Tensor("Neg:0", shape=(), dtype=floa